In [1]:
import pandas as pd
from pathlib import Path
import datetime

In [2]:
data_file = Path("Resources/afluenciastc_simple_08_2023.csv")

In [3]:
influx_df = pd.read_csv(data_file)

In [4]:
influx_df = influx_df.astype({"fecha": "datetime64"})
influx_df.dtypes

fecha        datetime64[ns]
anio                  int64
mes                  object
linea                object
estacion             object
afluencia             int64
dtype: object

In [5]:
influx_df = influx_df.rename(columns={'fecha': 'date', 'anio': 'year',
                                      'mes': 'month', 'linea': 'line',
                                      'estacion': 'station', 'afluencia': 'influx'})

In [6]:
influx_df['day'] = influx_df['date'].dt.day

In [7]:
influx_df['weekday'] = influx_df['date'].dt.day_of_week

In [8]:
day_off_path = Path('Resources\dias_no_habiles.csv')
day_off_df = pd.read_csv(day_off_path)

In [9]:
day_off_df = day_off_df.apply(pd.to_datetime)
day_off_df = day_off_df.stack().reset_index(drop=True)
influx_df['day_off'] = influx_df['date'].isin(day_off_df)

In [10]:
influx_df.head()

,date,year,month,line,station,influx,day,weekday,day_off
0,2010-01-01,2010,Enero,Línea 1,Zaragoza,20227,1,4,True
1,2010-01-01,2010,Enero,Línea 1,Isabel la Católica,6487,1,4,True
2,2010-01-01,2010,Enero,Línea 1,Moctezuma,10304,1,4,True
3,2010-01-01,2010,Enero,Línea 1,Pino Suárez,8679,1,4,True
4,2010-01-01,2010,Enero,Línea 1,Gómez Farías,19499,1,4,True


In [11]:
stations_group = influx_df[['station', 'line']].copy()

In [12]:
stations_group = stations_group.drop_duplicates(subset=['station', 'line'])
intersections_df = stations_group.groupby(
    'station')['line'].count().reset_index()
intersections_df = intersections_df.rename(columns={'line': 'ints'})

In [13]:
influx_df = pd.merge(influx_df, intersections_df, on='station', how='left')

In [14]:
def get_season(date):
    if (date.month >= 3 and date.month <= 5):
        return 'Spring'
    elif (date.month >= 6 and date.month <= 8):
        return 'Summer'
    elif (date.month >= 9 and date.month <= 11):
        return 'Autumn'
    else:
        return 'Winter'

In [15]:
influx_df['season'] = influx_df['date'].apply(get_season)

In [16]:
def Summer_vacations(date):
    spring_2019 = (date >= pd.Timestamp('2019-06-24')
                   ) and (date <= pd.Timestamp('2019-08-26'))
    spring_2020 = (date >= pd.Timestamp('2020-07-06')
                   ) and (date <= pd.Timestamp('2020-08-24'))
    spring_2021 = (date >= pd.Timestamp('2021-07-09')
                   ) and (date <= pd.Timestamp('2021-08-30'))
    spring_2022 = (date >= pd.Timestamp('2022-07-28')
                   ) and (date <= pd.Timestamp('2022-08-29'))

    return spring_2019 or spring_2020 or spring_2021 or spring_2022

In [17]:
influx_df['summer_vacations'] = influx_df['date'].apply(Summer_vacations)

In [18]:
stations_list = ['Zaragoza', 'Isabel la Católica', 'Moctezuma', 'Pino Suárez', 'Gómez Farías', 'Deportivo 18 de Marzo', 'La Villa/Basílica', 'Pantitlán', 'Aculco', 'Velódromo', 'Autobuses del Norte', 'Misterios',
                 'Constituyentes', 'Refinería', 'Etiopía/Plaza de la Transparencia', 'Polanco', 'Canal del Norte', 'Bondojito', 'Santa Anita', 'Popotla', 'General Anaya', 'División del Norte', 'Zapata',
                 'Fray Servando', 'Hangares', 'El Rosario', 'Ferrería/Arena Ciudad de México', 'Instituto del Petróleo', 'Coyuya', 'Apatlaco', 'La Viga', 'Chabacano', 'San Juan de Letrán', 'Barranca del Muerto',
                 'Garibaldi/Lagunilla', 'Buenavista', 'Guerrero', 'Oceanía', 'Nezahualcóyotl', 'Revolución', 'Indios Verdes', 'Hidalgo', 'Colegio Militar', 'Normal', 'San Cosme', 'Juárez', 'Olímpica', 'Ciudad Azteca',
                 'Tlaltenco', 'Insurgentes Sur', 'Culhuacán', 'Calle 11', 'Chilpancingo', 'Peñón Viejo', 'Canal de San Juan', 'Lomas Estrella', 'San Andrés Tomatlán', 'Ermita', 'Xola', 'Salto del Agua', 'Balderas',
                 'Portales', 'Eduardo Molina', 'Candelaria', 'Aragón', 'Tacubaya', 'San Pedro de los Pinos', 'Camarones', 'Doctores', 'San Antonio', 'Mixcoac', 'Tacuba', 'Martín Carrera', 'Auditorio', 'Aquiles Serdán',
                 'San Joaquín', 'Morelos', 'San Antonio Abad', 'Viaducto', 'Norte 45', 'Vallejo', 'Tezozómoc', 'UAM-Azcapotzalco', 'Bellas Artes', 'Cuitláhuac', 'Terminal Aérea', 'Eugenia', 'Observatorio',
                 'Cuatro Caminos', 'Panteones', 'Copilco', 'Centro Médico', 'Universidad', 'Lindavista', 'Coyoacán', 'Jamaica', 'La Raza', 'Viveros/Derechos Humanos', 'Miguel Ángel de Quevedo', 'Chapultepec', 'Obrera',
                 'Allende', 'Zócalo/Tenochtitlan', 'Nativitas', 'Boulevard Puerto Aéreo', 'Sevilla', 'Villa de Cortés', 'Consulado', 'Talismán', 'Valle Gómez', 'Politécnico', 'Niños Héroes', 'Hospital General',
                 'Tasqueña', 'Potrero', 'Tlatelolco', 'Balbuena', 'Tepito', 'Merced', 'Insurgentes', 'Juanacatlán', 'Plaza Aragón', 'Guelatao', 'Lagunilla', 'La Paz', 'Iztapalapa', 'Patriotismo', 'Cerro de la Estrella',
                 'Zapotitlán', 'Nopalera', 'Olivos', 'Río de los Remedios', 'Múzquiz', 'Atlalilco', 'Tezonco', 'Impulsora', 'Escuadrón 201', 'Periférico Oriente', 'Iztacalco', 'Tláhuac', 'UAM-I', 'Ecatepec', 'Acatitla',
                 'Hospital 20 de Noviembre', 'Eje Central', 'Parque de los Venados', 'San Lázaro', 'Romero Rubio', 'Villa de Aragón', 'Tepalcates', 'Agrícola Oriental', 'Deportivo Oceanía', 'Puebla', 'Ciudad Deportiva',
                 'Mixiuhca', 'Bosque de Aragón', 'Santa Marta', 'Los Reyes', 'Constitución de 1917', 'Cuauhtémoc', 'Mexicaltzingo', 'Ricardo Flores Magón', 'Lázaro Cárdenas']

district_list = ['Venustiano Carranza', 'Cuauhtémoc', 'Venustiano Carranza', 'Cuauhtémoc', 'Venustiano Carranza', 'Gustavo A. Madero', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'Iztacalco', 'Gustavo A. Madero',
                 'Cuauhtémoc', 'Miguel Hidalgo', 'Azcapotzalco', 'Benito Juárez', 'Miguel Hidalgo', 'Cuauhtémoc', 'Gustavo A. Madero', 'Iztacalco', 'Miguel Hidalgo', 'Benito Juárez', 'Benito Juárez', 'Benito Juárez',
                 'Venustiano Carranza', 'Venustiano Carranza', 'Azcapotzalco', 'Azcapotzalco', 'Gustavo A. Madero', 'Iztacalco', 'Iztacalco', 'Iztacalco', 'Cuauhtémoc', 'Cuauhtémoc', 'Álvaro Obregón', 'Cuauhtémoc',
                 'Cuauhtémoc', 'Cuauhtémoc', 'Venustiano Carranza', 'Venustiano Carranza', 'Benito Juárez', 'Gustavo A. Madero', 'Cuauhtémoc', 'Miguel Hidalgo', 'Cuauhtémoc', 'Cuauhtémoc', 'Cuauhtémoc', 'Benito Juárez',
                 'Ecatepec', 'Tláhuac', 'Benito Juárez', 'Iztapalapa', 'Iztapalapa', 'Benito Juárez', 'Tláhuac', 'Iztacalco', 'Iztapalapa', 'Iztapalapa', 'Iztapalapa', 'Benito Juárez', 'Cuauhtémoc', 'Cuauhtémoc',
                 'Benito Juárez', 'Gustavo A. Madero', 'Venustiano Carranza', 'Gustavo A. Madero', 'Miguel Hidalgo', 'Álvaro Obregón', 'Azcapotzalco', 'Cuauhtémoc', 'Tláhuac', 'Álvaro Obregón', 'Miguel Hidalgo',
                 'Gustavo A. Madero', 'Miguel Hidalgo', 'Azcapotzalco', 'Miguel Hidalgo', 'Venustiano Carranza', 'Cuauhtémoc', 'Benito Juárez', 'Gustavo A. Madero', 'Gustavo A. Madero', 'Azcapotzalco', 'Azcapotzalco',
                 'Cuauhtémoc', 'Azcapotzalco', 'Venustiano Carranza', 'Benito Juárez', 'Miguel Hidalgo', 'Miguel Hidalgo', 'Azcapotzalco', 'Coyoacán', 'Cuauhtémoc', 'Coyoacán', 'Gustavo A. Madero', 'Coyoacán',
                 'Venustiano Carranza', 'Gustavo A. Madero', 'Coyoacán', 'Coyoacán', 'Miguel Hidalgo', 'Cuauhtémoc', 'Cuauhtémoc', 'Cuauhtémoc', 'Benito Juárez', 'Venustiano Carranza', 'Benito Juárez', 'Benito Juárez',
                 'Venustiano Carranza', 'Gustavo A. Madero', 'Venustiano Carranza', 'Gustavo A. Madero', 'Benito Juárez', 'Cuauhtémoc', 'Coyoacán', 'Gustavo A. Madero', 'Cuauhtémoc', 'Venustiano Carranza', 'Cuauhtémémoc',
                 'Cuauhtémoc', 'Cuauhtémoc', 'Cuajimalpa', 'Ecatepec', 'Iztapalapa', 'Cuauhtémoc', 'La Paz (Estado de México)', 'Iztapalapa', 'Benito Juárez', 'Iztapalapa', 'Tláhuac', 'Tláhuac', 'Tláhuac',
                 'Gustavo A. Madero', 'Gustavo A. Madero', 'Iztapalapa', 'Iztapalapa', 'Nezahualcóyotl', 'Nezahualcóyotl', 'Nezahualcóyotl', 'Iztacalco', 'Tláhuac', 'Iztapalapa', 'Ecatepec', 'Iztapalapa', 'Álvaro Obregón',
                 'Cuauhtémoc', 'Benito Juárez', 'Cuauhtémoc', 'Venustiano Carranza', 'Gustavo A. Madero', 'Venustiano Carranza', 'Iztacalco', 'Venustiano Carranza', 'Venustiano Carranza', 'Iztacalco', 'Venustiano Carranza',
                 'Nezahualcóyotl', 'Iztapalapa', 'La Paz (Estado de México)', 'Tláhuac', 'Cuauhtémoc', 'Iztapalapa', 'Cuauhtémoc', 'Cuauhtémoc']

In [19]:
station_district_df = pd.DataFrame(
    {"station": stations_list, "district": district_list})

In [20]:
influx_df = pd.merge(influx_df, station_district_df, on='station', how='left')

In [21]:
influx_df.columns

Index(['date', 'year', 'month', 'line', 'station', 'influx', 'day', 'weekday',
       'day_off', 'ints', 'season', 'summer_vacations', 'district'],
      dtype='object')

In [22]:
date_df = influx_df[['date', 'day', 'month', 'year', 'weekday',
                     'day_off', 'season', 'summer_vacations']].copy()

date_df = date_df.drop_duplicates(subset=['date']).reset_index(drop=True)

date_df.to_csv('Outputs/dates.csv')

In [23]:
coordinates_path = ('Resources/estacion-coordenadas.csv')
coordinates_df = pd.read_csv(coordinates_path)

In [24]:
coordinates_df.columns = ['station', 'lat', 'long', 'a', 'b', 'c', 'd', 'e']
coordinates_df = coordinates_df.drop(['a', 'b', 'c', 'd', 'e'], axis=1)
new_row = ['Zaragoza', '19.4130896', '-99.08234751']
coordinates_df.loc[162] = new_row

coordinates_df = coordinates_df.drop(
    coordinates_df.index[[163, 164, 165, 166, 167, 168]], axis=0
)

In [25]:
influx_df = pd.merge(influx_df, coordinates_df, on='station', how='left')

In [26]:
stations_df = influx_df[['station', 'district', 'ints', 'lat', 'long']]
stations_df = stations_df.drop_duplicates(
    subset=['station']).reset_index(drop=True)

In [ ]:
stations_df.to_csv('Outputs/stations.csv')